# Obligatory assignment - **MEK4250** - *Trym Erik Nielsen*

## Required imports

In [ ]:
from dolfin import *
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display
%matplotlib inline

## Boundary class definitions

In [ ]:
class Right(SubDomain):
    def inside(self, x, on_boundary, eps=1.e-14):
        return x[0] > 1 - eps


class Left(SubDomain):
    def inside(self, x, on_boundary, eps=1.e-14):
        return x[0] < eps

## Problem specific solver functions

In [ ]:
def solver_prob_1(N=8, deg=1, k=1):
    mesh = UnitSquareMesh(N, N)
    V = FunctionSpace(mesh, 'P', deg)

    #numerical and analytical expressions
    u_e = Expression('sin(pi*k*x[0])*cos(pi*k*x[1])', degree=deg, k=k)
    u_num = Function(V)

    #Boundary conditions
    bcs = [DirichletBC(V, Constant(0.0), Left()),
           DirichletBC(V, Constant(1.0), Right())]

    u = TrialFunction(V)
    v = TestFunction(V)

    f = Constant(1.0)
    a = dot(grad(u), grad(v)) * dx
    L = f * v * dx

    u = Function(V)
    solve(a == L, u, bcs)
    
    return mesh, u, V


def solver_prob_2(N=8, mu=1, deg=1, galerkin=False):
    
    mesh = UnitSquareMesh(N, N)
    V = FunctionSpace(mesh, 'P', deg)
    u_num = Function(V)
    
    bcs = [
        DirichletBC(V, Constant(0.0), Left()),
        DirichletBC(V, Constant(1.0), Right()),
    ]
    
    u = TrialFunction(V)
    v = TestFunction(V)
    
    if galerkin:
        beta = 0.5 * mesh.hmin()
        v  = v + beta * v.dx(0)
    
    f = Constant(0.0)
    g = Constant(0.0)    
    
    a = mu * inner(grad(u), grad(v)) * dx + u.dx(0) * v * dx
    L = f * v * dx + g * v * ds
    
    solve(a==L, u_num, bcs)
    
    return u_num, V, mesh

## Problem functions

We can now define problem functions that call our problem specific solvers. We use pandas dataframes to display the errornorms for different frequencies and mesh refinements using the FEniCS built in errornorm function. Since errornorm does not support the $L_\infty$ errornorm, we project our expression for the exact solution onto our subspace and take the maximum of the absolute value of the difference with the numerical solution. 

We also define the error fitting function that creates a linear best-fit between our error norms $L_2$ and $H_1$

In [ ]:
def error_fit(L_2, H_1):
    N = L_2.index.values
    param = L_2.columns.values
    best_fit_coeffs = pd.DataFrame(index=param, columns=['alpha', 'C_alpha', 'beta', 'C_beta'])
    h_log = [np.log(1.0 / n) for n in N]
    L_2_log = L_2.applymap(np.log)
    H_1_log = H_1.applymap(np.log)
    
    plt.plot(h_log, L_2_log, 'r', h_log, H_1_log, 'b')
    plt.title('LogLog plot of error norm VS cell size')
    plt.xlabel('log(h)')
    plt.ylabel('log(L_2) / Log(H_1)')
    plt.show()
    
    for p in param:
        L_2_fit = np.polyfit(h_log, L_2_log[p], deg=1)
        H_1_fit = np.polyfit(h_log, H_1_log[p], deg=1)
        
        L_2_fit[1] = np.exp(L_2_fit[1])
        H_1_fit[1] = np.exp(H_1_fit[1])
    
        best_fit_coeffs.loc[p] = list(L_2_fit) + list(H_1_fit)


    return best_fit_coeffs   

def prob_1(deg=1):
    
    N = [8, 16, 32, 64]
    freqs = [1, 2, 4,8]
    L_2_error = pd.DataFrame(index=N, columns=freqs)
    L_inf_error = pd.DataFrame(index=N, columns=freqs)
    H_1_error = pd.DataFrame(index=N, columns=freqs)
    
    
    for n in N:
        for freq in freqs:
                mesh,u_num,V = solver_prob_1(N=n, deg=deg, k=freq)
                u_ex = Expression('sin(k*pi*x[0])*cos(k*pi*x[1])', k=freq, degree=deg) 
                
                L_2 = errornorm(u_ex, u_num, 'l2', degree_rise=1)
                H_1 = errornorm(u_ex, u_num, 'H1', degree_rise=1)
                
                u_ex_values = interpolate(u_ex, V)
                u_ex_vector = u_ex_values.vector()
                u_num_vector = u_num.vector()
                L_inf = max(np.abs(u_ex_vector - u_num_vector))

                L_2_error.at[n, freq] = L_2
                L_inf_error.at[n, freq] = L_inf
                H_1_error.at[n, freq] = H_1
    
    
    print('----- problem 1 - part b -------' + '\n')
    print('L2 error norm' + '\n')
    display(L_2_error)
    print('L_inf error norm' + '\n')
    display(L_inf_error)
    print('H1 error norm' + '\n')
    display(H_1_error)
    print('\n\n\n')
    print('----- problem 1 - part c -------' + '\n')
    print('Best fit parameters')
    display(error_fit(L_2_error, H_1_error))
    
    #plot highest accuracy mesh and u  
    if deg > 1:
        p = plot(u_num)
        p.set_cmap('plasma')
        p.set_clim(0.0, 1.0)
        plt.title('u solution')
        plt.colorbar(p)
        plt.show()

        
def prob_2(deg=1, galerkin=False):
    mu_values = [0.1, 0.3, 1.0]
    N = [8, 16, 32, 64]

    L_2_error = pd.DataFrame(index=N, columns=mu_values)
    L_inf_error = pd.DataFrame(index=N, columns=mu_values)
    H_1_error = pd.DataFrame(index=N, columns=mu_values)
    
    for mu in mu_values:
        for n in N:
            u_num, V, omega = solver_prob_2(N=n, mu=mu, deg=deg, galerkin=galerkin)
            u_ex = Expression('(exp(1 / mu * x[0]) - 1) / (exp(1 / mu) - 1)', mu=mu, degree=deg)

            L_2 = errornorm(u_ex, u_num, 'L2', degree_rise=3)
            H_1 = errornorm(u_ex, u_num, 'H1', degree_rise=3)
            u_ex_values = interpolate(u_ex, V)
            u_ex_vector = u_ex_values.vector()
            u_num_vector = u_num.vector()
            L_inf = max(np.abs(u_ex_vector - u_num_vector))
            
            L_2_error.at[n, mu] = L_2
            L_inf_error.at[n, mu] = L_inf
            H_1_error.at[n, mu] = H_1
            
    print('----- problem 2 - part b -------' + '\n')
    print('L2 error norm' + '\n')
    display(L_2_error)
    print('L_inf error norm' + '\n')
    display(L_inf_error)
    print('H1 error norm' + '\n')
    display(H_1_error)
    print('\n\n\n')
    print('----- problem 2 - part c -------' + '\n')
    print('Best fit parameters')
    display(error_fit(L_2_error, H_1_error))
            
    if deg >= 1:
        p = plot(u_num)
        p.set_cmap('plasma')
        p.set_clim(0.0, 1.0)
        plt.title('u solution')
        plt.colorbar(p)
        plt.show()

### $H^P$ norm of $u$

The $H^P$ norm of the function $u$ can be expressed as
$$\vert \vert \, u \, \vert \vert_P = \left( \sum^{}_{|\alpha|\leq p} \int_\Omega
    \big(\frac{\partial^{|\alpha|} u}{\partial {x}^\alpha}\big)^2 \, dx
\right)^{1/2}$$ 

with $\alpha = (i,j)$ in the two dimensional Cartesian space. Using our assumed $u$, and the fact that $f = - \Delta u  = 2\pi^2 k^2 u$, we find

$$\vert \vert \, u \, \vert \vert_P = \frac{1}{2} \Big(\sum^{}_{|\alpha| \leq p} (k\pi)^{2|\alpha|}\Big)^{1/2}$$

Since $\int_0^1 \sin(\pi k x) = \int_0^1 \cos(\pi k y) = \frac{1}{2}$

### Problem 1, Lagrange elements order 1

In [ ]:
prob_1()

### Problem 1, Lagrange elements order 2

In [ ]:
prob_1(2)

### Problem 2, without SUPG stablization, Lagrange elements order 2

In [ ]:
prob_2(2)

### Problem 2, with SUPG stabalization, Lagrange elements order 1 

In [ ]:
prob_2(1, galerkin=True)

## Problem 2 - discussion

**Analytical Solution**  

Our boundary value problem reads:
$$ -\mu\Delta + u_x = 0 \; in \; \Omega$$

with Dirichlet BC's;

$$u = 0 \; for \; x = 0$$

$$u = 1 \; for \; x = 1$$

and Neumann BC's:
$$\frac{\partial{u}}{\partial{n}} = 0 \; for \; y=0 \; and \; y=1$$


We can solve this analytically by the method of seperation of variables, we let $u(x,y) = g(x)h(y)$ and plugging into our original equation and dividing by $\mu$ gives us the two equations

$$g''(x) - \frac{1}{\mu} - C_0 g(x) = 0 \,,$$
$$h''(y) + C_0 h(y) = 0 \,,$$


Solving for h and using the above Neumann b.c, we obtain:
$$h(y) = C_1 \cos(n \pi y), \; for \: n=0,1,2, ..$$

Choosing $n = 0$ we have $C_0 = 0$, and our expression for $h(y)$ becomes

$$h(y) = C_1$$


with $n = 0$, our expression for $g(x)$ can be expressed as

$$g''(x) - \frac{1}{\mu} g'(x) = 0$$

which gives the expression for $g(x)$

$$g(x) = C_2 e^{\frac{1}{\mu} x} + C_3$$

using the dirichlet conditions, we can solve for $C_2$ and $C_3$ we obtain the analytical expression for $u$ as 

$$u(x,y) = g(x)h(y) = \frac{e^{\frac{1}{\mu} x} - 1}{e^{\frac{1}{\mu}} - 1}$$


**Numerical error**  

Problem 2 includes a convective $u_x$ term, and we can therefore reasonably expect a upwinding scheme to stablize the solution of our system. Both the $L_1$ and $H^1$ tend to zero linearly with the mesh size for both Lagrange elements of degree and 2. However, the $H^1$ norm tends to zero faster than the $L_1$ norm for lagrange elements of order 1. We can also see from the $C_\alpha$ and $C_\beta$ terms for upwinding leads to more accurate results for small $\mu$ where the $u_x$ term dominates and low values of $N$